In [21]:
import numpy as np
import pandas as pd
import re
import io
from urllib.request import urlopen
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [142]:
# qr_id = 4820051240233
qr_id = 4820001448603

qr_url = f'https://listex.info/search/?q={qr_id}&type=goods'

qr_html_text = requests.get(qr_url).text
qr_soup = BeautifulSoup(qr_html_text, 'lxml')

In [143]:
for link in qr_soup.find_all('a', class_='link-gray'):
    href = re.match(r'.product.', link.get('href'))
    if href:
        prod_link = 'https://listex.info/' + link.get('href')

In [145]:
prod_html_text = requests.get(prod_link).text
prod_soup = BeautifulSoup(prod_html_text, 'lxml')

In [294]:
list_text = prod_soup.find('p', class_="product-specifications-title").get_text().split()[1:]
prod_name = ' '.join(list_text)

In [301]:
log_params = str(prod_soup.find_all('table')[1])
package = log_params.split('Материал транспортной тары')[1].replace('</th>','').replace('\n','').replace('<tr>','').replace('<th>','').replace('<td>','').replace('</td>','').replace('</tr>','').split('Белки, г/100г')[0].replace(' ','').replace('\t', '').replace('</table>','')

In [300]:
log_params2 = str(prod_soup.find_all('table')[2])
mass = log_params2.split('Вес нетто, пустой box, кг')[1].replace('</th>','').replace('\n','').replace('<tr>','').replace('<th>','').replace('<td>','').replace('</td>','').replace('</tr>','').split('Белки, г/100г')[0].replace(' ','').replace('\t', '').replace('</table>','')

In [370]:
if re.findall(r'Питательные характеристики', str(prod_soup)):
    nutritional = str(prod_soup.find_all('table')[4])

    fats = nutritional.split('Жиры, г/100г')[1].replace('</th>','').replace('\n','').replace('<tr>','').replace('<th>','').replace('<td>','').replace('</td>','').replace('</tr>','').split('Белки, г/100г')[0].replace(' ','').replace('\t', '')
    proteins = nutritional.split('Белки, г/100г')[1].replace('</th>','').replace('\n','').replace('<tr>','').replace('<th>','').replace('<td>','').replace('</td>','').replace('</tr>','').split('Углеводы, г/100г')[0].replace(' ','').replace('\t', '')
    carbohydrates = nutritional.split('Углеводы, г/100г')[1].replace('</th>','').replace('\n','').replace('<tr>','').replace('<th>','').replace('<td>','').replace('</td>','').replace('</tr>','').split('Калорийность, ккал/100г')[0].replace(' ','').replace('\t', '')
    calories = nutritional.split('Калорийность, ккал/100г')[1].replace('</th>','').replace('\n','').replace('<tr>','').replace('<th>','').replace('<td>','').replace('</td>','').replace('</tr>','').split('кДж/100г')[0].replace(' ','').split('<')[0].replace('\t', '')
    kdg = string.split('кДж/100г')[1].replace('</th>','').replace('\n','').replace('<tr>','').replace('<th>','').replace('<td>','').replace('</td>','').replace('</tr>','').replace(' ','').split('<')[0].replace('\t', '')

else:
    fats = 0
    proteins = 0
    carbohydrates = 0
    calories = 0
    kdg = 0

In [344]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>|&ndash; ')
    return re.sub(clean, '', text)

def replase_to_bool(val):
    if val == 'НЕТ':
        ans = False
    elif val == 'ДА':
        ans = True
    return ans

main = str(prod_soup.find_all('table')[6])

clean_text = remove_html_tags(main).replace('\n','').replace('\t','')
clean_text = clean_text.split('  ')

ans = []
for i in clean_text:
    if len(i) !=0:
        ans.append(i)

gmo = replase_to_bool(ans[1])
organic = replase_to_bool(ans[3])
vegetarian = replase_to_bool(ans[5])
vegan = replase_to_bool(ans[7])

In [366]:
description = str(prod_soup.find_all('table')[10].td).split()
components = ' '.join(description).replace('<td> ', '').replace(' </td>', '')

In [375]:
dict_ = {
    'barcode': qr_id,
    'name': prod_name,

    'proteins': float(proteins), 
    'fats': float(fats),
    'carbohydrates': float(carbohydrates),
    'calories': float(calories),

    'mass': float(mass),

    'package': package,

    'is_gmo': gmo,
    'is_organic': organic,

    'is_vegetarian': vegetarian,
    'is_vegan': vegan,

    'components': components
}

In [376]:
dict_ = {
    'barcode': qr_id,
    'name': prod_name,

    'proteins': float(proteins), 
    'fats': float(fats),
    'carbohydrates': float(carbohydrates),
    'calories': float(calories),

    'mass': float(mass),

    'package': package,

    'is_gmo': gmo,
    'is_organic': organic,

    'is_vegetarian': vegetarian,
    'is_vegan': vegan,

    'components': components
}

{'barcode': 4820001448603,
 'calories': 48.0,
 'carbohydrates': 11.5,
 'components': 'Мультифруктовый сок концентрированный (соки ананаса, апельсина, мандарина, пюре банана, соки лимона и маракуйи) (33%), яблочный сок концентрированный (17%), сахарный сироп, регулятор кислотности лимонная кислота, витамины: провитамин А (β-каротин), Е, В1, В2, РР, В6, С, В5, В9, Н, натуральные ароматизаторы "Тропик"',
 'fats': 0.0,
 'is_gmo': False,
 'is_organic': False,
 'is_vegan': True,
 'is_vegetarian': True,
 'mass': 0.156,
 'name': 'Нектар неосветленный Мультивитамин Sandora т/п 500мл.',
 'package': 'КАРТОННАЯ',
 'proteins': 0.0}